# *Introduction to Decision Trees*

### Overview of the Dataset

In [17]:
import pandas as pd
# Set index_col to False to avoid pandas thinking that the first column is row indexes (it's age)

income = pd.read_csv('income.csv', index_col=False)
income.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,high_income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


### Converting Categorical Variables

In [18]:
col = pd.Categorical(income['workclass'])
income['workclass'] = col.codes
print(income['workclass'].head())

0    7
1    6
2    4
3    4
4    4
Name: workclass, dtype: int8


In [19]:
cols = ['education', 'marital_status', 'occupation', 'relationship', 'race', 
        'sex', 'native_country', 'high_income']

for item in cols:
    col = pd.Categorical(income[item])
    income[item] = col.codes
    
income.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,high_income
0,39,7,77516,9,13,4,1,1,4,1,2174,0,40,39,0
1,50,6,83311,9,13,2,4,0,4,1,0,0,13,39,0
2,38,4,215646,11,9,0,6,1,4,1,0,0,40,39,0
3,53,4,234721,1,7,2,6,0,2,1,0,0,40,39,0
4,28,4,338409,9,13,2,10,5,2,0,0,0,40,5,0


### Creating Splits

In [4]:
private_incomes = income[income['workclass'] == 4] 
public_incomes = income[income['workclass'] != 4]

### Overview of Data Set Entropy

In [5]:
import math 
values = income['high_income'].value_counts()
total = income.shape[0]
income_entropy = -(values[0]/total * math.log(values[0]/total, 2) + 
                   values[1]/total * math.log(values[1]/total, 2))
income_entropy

0.7963839552022132

### Information Gain

In [6]:
import numpy

def calc_entropy(column):
    """
    Calculate entropy given a pandas series, list, or numpy array.
    """
    # Compute the counts of each unique value in the column
    counts = numpy.bincount(column)
    # Divide by the total column length to get a probability
    probabilities = counts / len(column)
#     print('probabilities', probabilities)
    
    # Initialize the entropy to 0
    entropy = 0
    # Loop through the probabilities, and add each one to the total entropy
    for prob in probabilities:
        if prob > 0:
            entropy += prob * math.log(prob, 2)
    
    return -entropy

# Verify that our function matches our answer from earlier
# entropy = calc_entropy([1,1,0,0,1])
# print(entropy)

# information_gain = entropy - ((.8 * calc_entropy([1,1,0,0])) + (.2 * calc_entropy([1])))
# print(information_gain)

median_age = income['age'].median()
left_split = income[income['age']<= median_age]
len_left_split = len(left_split)

right_split = income[income['age'] > median_age]
len_right_split = len(right_split)

total = income.shape[0]

age_information_gain = income_entropy - (
    ((len_left_split/total) * calc_entropy(left_split['high_income'])) + 
    ((len_right_split/total) * calc_entropy(right_split['high_income'])))

age_information_gain

0.047028661304691965

### Finding the Best Split

In [7]:
def calc_information_gain(data, split_name, target_name):
    """
    Calculate information gain given a data set, column to split on, and target
    """
    # Calculate the original entropy
    original_entropy = calc_entropy(data[target_name])
    
    # Find the median of the column we're splitting
    column = data[split_name]
    median = column.median()
    
    # Make two subsets of the data, based on the median
    left_split = data[column <= median]
    right_split = data[column > median]
    
    # Loop through the splits and calculate the subset entropies
    to_subtract = 0
    for subset in [left_split, right_split]:
        subset_size = (subset.shape[0] / data.shape[0]) 
        to_subtract += subset_size * calc_entropy(subset[target_name])
    
    # Return information gain
    return original_entropy - to_subtract

# Verify that our answer is the same as on the last screen
print(calc_information_gain(income, "age", "high_income"))

columns = ["age", "workclass", "education_num", "marital_status", "occupation", "relationship", 
           "race", "sex", "hours_per_week", "native_country"]

information_gains = []

for item in columns:
    information_gains.append(calc_information_gain(income, item, 'high_income'))
    
print('\nInformation Gains', information_gains)

highest_gain_value = max(information_gains)  
print('\nHighest Information Gain Value', highest_gain_value)

highest_gain = columns[information_gains.index(highest_gain_value)]
highest_gain

0.047028661304691965

Information Gains [0.047028661304691965, 0.006810984054396618, 0.06501298413277423, 0.1114272573715438, 0.0015822303843424645, 0.04736241665026941, 0.0, 0.0, 0.04062246867123487, 0.00013457344495848567]

Highest Information Gain Value 0.1114272573715438


'marital_status'

# *Building a Decision Tree*
### Determining the Column to Split On

In [8]:
def find_best_column(data, target_name, columns):
    # Fill in the logic here to automatically find the column in columns to split on
    # data is a dataframe
    # target_name is the name of the target variable
    # columns is a list of potential columns to split on
    information_gains = []
    
    for item in columns:
        information_gains.append(calc_information_gain(data, item, target_name))
    
    highest_gain_value = max(information_gains)
    
    return columns[information_gains.index(highest_gain_value)]

# A list of columns to potentially split income with
columns = ["age", "workclass", "education_num", "marital_status", "occupation", "relationship", 
           "race", "sex", "hours_per_week", "native_country"]

income_split = find_best_column(income, 'high_income', columns)
print(income_split)

marital_status


### Creating a Simple Recursive Algorithm

In [9]:
# We'll use lists to store our labels for nodes (when we find them)
# Lists can be accessed inside our recursive function, whereas integers can't.  
# Look at the python missions on scoping for more information on this topic
label_1s = []
label_0s = []

def id3(data, target, columns):
    # The pandas.unique method will return a list of all the unique values in a series
    unique_targets = pd.unique(data[target])
    
    if len(unique_targets) == 1:
        # Insert code here to append 1 to label_1s or 0 to label_0s, based on what we should label the node
        # See lines 2 and 3 in the algorithm
        unique = unique_targets[0]
        if unique == 1:
            label_1s.append(unique)
        else:
            label_0s.append(unique)
        
        # Returning here is critical -- if we don't, the recursive tree will never finish, and run forever
        # See our example above for when we returned
        return
    
    # Find the best column to split on in our data
    best_column = find_best_column(data, target, columns)
    # Find the median of the column
    column_median = data[best_column].median()
    
    # Create the two splits
    left_split = data[data[best_column] <= column_median]
    right_split = data[data[best_column] > column_median]
    
    # Loop through the splits and call id3 recursively
    for split in [left_split, right_split]:
        # Call id3 recursively to process each branch
        id3(split, target, columns)
    
# Create the data set that we used in the example on the last screen
data = pd.DataFrame([
    [0,20,0],
    [0,60,2],
    [0,40,1],
    [1,25,1],
    [1,35,2],
    [1,55,1]
    ])
# Assign column names to the data
data.columns = ["high_income", "age", "marital_status"]

# Call the function on our data to set the counters properly
id3(data, "high_income", ["age", "marital_status"])

print(label_0s)
print(label_1s)

[0, 0, 0]
[1, 1, 1]


### Storing the Tree

In [10]:
# Create a dictionary to hold the tree  
# It has to be outside of the function so we can access it later
tree = {}

# This list will let us number the nodes  
# It has to be a list so we can access it inside the function
nodes = []

def id3(data, target, columns, tree):
    unique_targets = pd.unique(data[target])
    
    # Assign the number key to the node dictionary
    nodes.append(len(nodes) + 1)
    tree["number"] = nodes[-1]

    if len(unique_targets) == 1:
        # Insert code here that assigns the "label" field to the node dictionary
        tree['label'] = unique_targets[0]
        return
    
    best_column = find_best_column(data, target, columns)
    column_median = data[best_column].median()
    
    # Insert code here that assigns the "column" and "median" fields to the node dictionary
    tree['column'] = best_column
    tree['median'] = column_median
    
    left_split = data[data[best_column] <= column_median]
    right_split = data[data[best_column] > column_median]
    split_dict = [["left", left_split], ["right", right_split]]
    
    for name, split in split_dict:
        tree[name] = {}
        id3(split, target, columns, tree[name])

# Call the function on our data to set the counters properly
id3(data, "high_income", ["age", "marital_status"], tree)

tree

{'number': 1,
 'column': 'age',
 'median': 37.5,
 'left': {'number': 2,
  'column': 'age',
  'median': 25.0,
  'left': {'number': 3,
   'column': 'age',
   'median': 22.5,
   'left': {'number': 4, 'label': 0},
   'right': {'number': 5, 'label': 1}},
  'right': {'number': 6, 'label': 1}},
 'right': {'number': 7,
  'column': 'age',
  'median': 55.0,
  'left': {'number': 8,
   'column': 'age',
   'median': 47.5,
   'left': {'number': 9, 'label': 0},
   'right': {'number': 10, 'label': 1}},
  'right': {'number': 11, 'label': 0}}}

### Printing Labels for a more attractive Tree

In [11]:
def print_with_depth(string, depth):
    # Add space before a string
    prefix = "    " * depth
    # Print a string, and indent it appropriately
    print("{0}{1}".format(prefix, string))
    
    
def print_node(tree, depth):
    # Check for the presence of "label" in the tree
    if "label" in tree:
        # If found, then this is a leaf, so print it and return
        print_with_depth("Leaf: Label {0}".format(tree["label"]), depth)
        # This is critical -- without it, you'll get infinite recursion
        return
    # Print information about what the node is splitting on
    print_with_depth("{0} > {1}".format(tree["column"], tree["median"]), depth)
    
    # Create a list of tree branches
    branches = [tree["left"], tree["right"]]
        
    # Insert code here to recursively call print_node on each branch
    # Don't forget to increment depth when you pass it in
    for item in branches:
        print_node(item, depth+1)

print_node(tree, 0)

age > 37.5
    age > 25.0
        age > 22.5
            Leaf: Label 0
            Leaf: Label 1
        Leaf: Label 1
    age > 55.0
        age > 47.5
            Leaf: Label 0
            Leaf: Label 1
        Leaf: Label 0


### Making Predictions Automatically

In [12]:
def predict(tree, row):
    if "label" in tree:
        return tree["label"]
    
    column = tree["column"]
    median = tree["median"]
    
    # Insert code here to check whether row[column] is less than or equal to median
    # If it's less than or equal, return the result of predicting on the left branch of the tree
    if (row[column] <= median):
        return predict(tree['left'], row)
        
    # If it's greater, return the result of predicting on the right branch of the tree
    # Remember to use the return statement to return the result!
    else:
        return predict(tree['right'], row)

# Print the prediction for the first row in our data
print(predict(tree, data.iloc[0]))

0


### Making Multiple Predictions

In [13]:
new_data = pd.DataFrame([
    [40,0],
    [20,2],
    [80,1],
    [15,1],
    [27,2],
    [38,1]
    ])
# Assign column names to the data
new_data.columns = ["age", "marital_status"]

def batch_predict(tree, df):
    # Insert your code here
    return df.apply(lambda x: predict(tree, x), axis=1)

predictions = batch_predict(tree, new_data)
predictions

0    0
1    0
2    0
3    0
4    1
5    0
dtype: int64

# *Applying Decision Trees*
### Splitting the Data into Train and Test Sets

In [20]:
# Set a random seed so the shuffle is the same every time
numpy.random.seed(1)

# Shuffle the rows  
# This permutes the index randomly using numpy.random.permutation
# Then, it reindexes the dataframe with the result
# The net effect is to put the rows into random order
income = income.reindex(numpy.random.permutation(income.index))

train_max_row = math.floor(income.shape[0] * .8)
train = income.iloc[:train_max_row]
test = income.iloc[train_max_row:]

train.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,high_income
9646,62,6,26911,5,4,6,8,1,4,0,0,0,66,39,0
709,18,4,208103,1,7,4,8,2,4,1,0,0,25,39,0
7385,25,4,102476,9,13,4,5,3,4,1,27828,0,50,39,1
16671,33,4,511517,11,9,2,10,0,4,1,0,0,40,39,0
21932,36,4,292570,1,7,4,7,4,4,0,0,0,40,39,0


In [21]:
test.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,high_income
22528,30,4,329425,11,9,2,3,0,4,1,0,0,40,39,0
31546,35,4,48779,11,9,2,5,0,4,1,0,0,35,39,0
18701,34,2,209213,9,13,4,10,2,2,1,0,0,15,39,0
10957,44,4,173590,9,13,2,10,0,4,1,0,0,3,39,0
1931,50,0,156008,1,7,2,0,3,2,0,0,0,40,39,0


### Using Decision Trees with Scikit-Learn

In [23]:
from sklearn.tree import DecisionTreeClassifier

# A list of columns to train with
# We've already converted all columns to numeric
columns = ["age", "workclass", "education_num", "marital_status", "occupation", "relationship", 
           "race", "sex", "hours_per_week", "native_country"]

# Instantiate the classifier
# Set random_state to 1 to make sure the results are consistent
clf = DecisionTreeClassifier(random_state=1)

# We've already loaded the variable "income," which contains all of the income data
clf.fit(train[columns], train['high_income'])

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=1,
            splitter='best')

### Evaluating Error with AUC


In [25]:
from sklearn.metrics import roc_auc_score

clf = DecisionTreeClassifier(random_state=1)
clf.fit(train[columns], train["high_income"])

predictions = clf.predict(test[columns])
error = (roc_auc_score(test['high_income'], predictions))
print(error)

0.6934656324746192


### Computing Error on the Training Set

In [27]:
predictions = clf.predict(train[columns])
print(roc_auc_score(train['high_income'], predictions))

0.9471244501437455


### Reducing Overfitting with a Shallower Tree

In [29]:
# Decision trees model from the last screen
clf = DecisionTreeClassifier(random_state=1, min_samples_split = 13)
clf.fit(train[columns], train['high_income'])

train_auc = roc_auc_score(train['high_income'], clf.predict(train[columns]))
test_auc = roc_auc_score(test['high_income'], clf.predict(test[columns]))

print(train_auc)
print(test_auc)

0.8421431849275413
0.6995617145150872


### Tweaking parameters to adjust AUC

In [30]:
clf = DecisionTreeClassifier(random_state=1, min_samples_split=13, max_depth=7)

clf.fit(train[columns], train['high_income'])

train_auc = roc_auc_score(train['high_income'], clf.predict(train[columns]))
test_auc = roc_auc_score(test['high_income'], clf.predict(test[columns]))

print(train_auc)
print(test_auc)

0.748037708309209
0.7436344996725136


### Tweaking Tree Depth to adjust AUC

In [31]:
clf = DecisionTreeClassifier(random_state=1, min_samples_split=100, max_depth=2)

clf.fit(train[columns], train['high_income'])

train_auc = roc_auc_score(train['high_income'], clf.predict(train[columns]))
test_auc = roc_auc_score(test['high_income'], clf.predict(test[columns]))

print(train_auc)
print(test_auc)

0.6624508042161483
0.6553138481876499


# *Introduction to Random Forests*

### Combining Model Predictions With Ensembles

In [34]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score

columns = ["age", "workclass", "education_num", "marital_status", "occupation", "relationship", "race", "sex", "hours_per_week", "native_country"]

clf = DecisionTreeClassifier(random_state=1, min_samples_leaf=2)
clf.fit(train[columns], train["high_income"])
print(roc_auc_score(test['high_income'], clf.predict(test[columns])))

clf2 = DecisionTreeClassifier(random_state=1, max_depth=5)
clf2.fit(train[columns], train["high_income"])
print(roc_auc_score(test['high_income'], clf2.predict(test[columns])))

0.6878964226062301
0.6759853906508785


### Combining our Predictions

In [36]:
predictions = clf.predict_proba(test[columns])[:,1]
predictions2 = clf2.predict_proba(test[columns])[:,1]

average_predictions = numpy.round((predictions + predictions2)/2)
print(roc_auc_score(test['high_income'], average_predictions))

0.7150846804038882


### Introducing Variation With Bagging

In [39]:
# We'll build 10 trees
tree_count = 10

# Each "bag" will have 60% of the number of original rows
bag_proportion = .6

predictions = []
for i in range(tree_count):
    # We select 60% of the rows from train, sampling with replacement
    # We set a random state to ensure we'll be able to replicate our results
    # We set it to i instead of a fixed value so we don't get the same sample in every loop
    # That would make all of our trees the same
    bag = train.sample(frac=bag_proportion, replace=True, random_state=i)
    
    # Fit a decision tree model to the "bag"
    clf = DecisionTreeClassifier(random_state=1, min_samples_leaf=2)
    clf.fit(bag[columns], bag["high_income"])
    
    # Using the model, make predictions on the test data
    predictions.append(clf.predict_proba(test[columns])[:,1])
    
average_predictions = numpy.round(numpy.sum(predictions, axis = 0)/tree_count)
print(roc_auc_score(test['high_income'], average_predictions))

0.7329963297474371


### Selecting Random Features

In [103]:
# Create the data set that we used two missions ago
data = pd.DataFrame([
    [0,4,20,0],
    [0,4,60,2],
    [0,5,40,1],
    [1,4,25,1],
    [1,5,35,2],
    [1,5,55,1]
    ])
data.columns = ["high_income", "employment", "age", "marital_status"]

# Set a random seed to make the results reproducible
numpy.random.seed(1)

# The dictionary to store our tree
tree = {}
nodes = []

# The function to find the column to split on
def find_best_column(data, target_name, columns):
    information_gains = []
    
    # Insert your code here
    random_columns = numpy.random.choice(columns, 2, replace=False)
    
    for col in random_columns:
        information_gain = calc_information_gain(data, col, "high_income")
        information_gains.append(information_gain)

    # Find the name of the column with the highest gain
    highest_gain_index = information_gains.index(max(information_gains))
    highest_gain = random_columns[highest_gain_index]
    return highest_gain

# The function to construct an ID3 decision tree
def id3(data, target, columns, tree):
    unique_targets = pd.unique(data[target])
    nodes.append(len(nodes) + 1)
    tree["number"] = nodes[-1]

    if len(unique_targets) == 1:
        if 0 in unique_targets:
            tree["label"] = 0
        elif 1 in unique_targets:
            tree["label"] = 1
        return
    
    best_column = find_best_column(data, target, columns)
    column_median = data[best_column].median()
    
    tree["column"] = best_column
    tree["median"] = column_median
    
    left_split = data[data[best_column] <= column_median]
    right_split = data[data[best_column] > column_median]
    split_dict = [["left", left_split], ["right", right_split]]
    
    for name, split in split_dict:
        tree[name] = {}
        id3(split, target, columns, tree[name])


# Run the ID3 algorithm on our data set and print the resulting tree
id3(data, "high_income", ["employment", "age", "marital_status"], tree)
print(tree)

{'number': 1, 'column': 'employment', 'median': 4.5, 'left': {'number': 2, 'column': 'age', 'median': 25.0, 'left': {'number': 3, 'column': 'marital_status', 'median': 0.5, 'left': {'number': 4, 'label': 0}, 'right': {'number': 5, 'label': 1}}, 'right': {'number': 6, 'label': 0}}, 'right': {'number': 7, 'column': 'marital_status', 'median': 1.0, 'left': {'number': 8, 'column': 'age', 'median': 47.5, 'left': {'number': 9, 'label': 0}, 'right': {'number': 10, 'label': 1}}, 'right': {'number': 11, 'label': 1}}}


### Random Subsets in Scikit-Learn

In [104]:
# We'll build 10 trees
tree_count = 10

# Each "bag" will have 60% of the number of original rows
bag_proportion = .6

predictions = []
for i in range(tree_count):
    # We select 60% of the rows from train, sampling with replacement
    # We set a random state to ensure we'll be able to replicate our results
    # We set it to i instead of a fixed value so we don't get the same sample every time
    bag = train.sample(frac=bag_proportion, replace=True, random_state=i)
    
    # Fit a decision tree model to the "bag"
    clf = DecisionTreeClassifier(random_state=1, min_samples_leaf=2,
                                splitter = 'random', max_features='auto')
    clf.fit(bag[columns], bag["high_income"])
    
    # Using the model, make predictions on the test data
    predictions.append(clf.predict_proba(test[columns])[:,1])

combined = numpy.sum(predictions, axis=0) / 10
rounded = numpy.round(combined)

print(roc_auc_score(test["high_income"], rounded))

0.7345958637997538


### Practice Putting it All Together

In [105]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=5, random_state=1, min_samples_leaf=2)

clf.fit(train[columns], train['high_income'])
print(roc_auc_score(test['high_income'], clf.predict(test[columns])))

0.7347461391939776


### Tweaking Parameters to increase Accuracy

In [106]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=150, random_state=1, min_samples_leaf=2)

clf.fit(train[columns], train["high_income"])

predictions = clf.predict(test[columns])
print(roc_auc_score(test["high_income"], predictions))

0.7379403213124711


### Reducing Overfitting

In [107]:
clf = DecisionTreeClassifier(random_state=1, min_samples_leaf=5)

clf.fit(train[columns], train["high_income"])

predictions = clf.predict(train[columns])
print(roc_auc_score(train["high_income"], predictions))

predictions = clf.predict(test[columns])
print(roc_auc_score(test["high_income"], predictions))

clf = RandomForestClassifier(n_estimators=150, random_state=1, min_samples_leaf=5)

clf.fit(train[columns], train["high_income"])

predictions = clf.predict(train[columns])
print(roc_auc_score(train["high_income"], predictions))

predictions = clf.predict(test[columns])
print(roc_auc_score(test["high_income"], predictions))

0.8192570489534683
0.7139325899284541
0.7917047295143252
0.7498874343962398
